In [1]:
import numpy as np
import pandas as pd
import os, json
import glob

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

import progressbar

import matplotlib.pyplot as plt

In [10]:
# dataframe = pd.read_json('../scraper/posts.json')

# posts_data = dataframe.drop(['post_id', 'thread_id', 'user_name', 'msg', 'reply_to'], axis=1)

pd.set_option('display.max_columns', None)

temp = pd.DataFrame()

path_to_json = './s3/*' 

json_pattern = os.path.join(path_to_json,'*.json')
file_list = glob.glob(json_pattern)

print("Loading JSON File")

result = []
for f in progressbar.progressbar(file_list):
    with open(f, "rb") as infile:
        result = result + json.load(infile)

posts_data = pd.DataFrame(result)

# i = 0
# for file in progressbar.progressbar(file_list):
#     data = pd.read_json(file, lines=True)
#     temp = temp.append(data, ignore_index = True)
#     i+=1
#     if i % 10000 == 0:
#         temp.to_pickle(f"./data_{i}.pkl")
    

# posts_data = temp
posts_data.to_pickle("data_final.pkl")
posts_data.head()

  0% (318 of 203966) |                   | Elapsed Time: 0:00:00 ETA:   0:01:04

Loading JSON File


100% (203966 of 203966) |################| Elapsed Time: 4:10:49 Time:  4:10:49


,root,thread_id,post_id,user_id,user_name,user_level,is_newbie,is_not_push_post,msg,like_count,dislike_count,create_time,reply_to
0,True,1791346,694ee0d54ccad4523d76ca52fb8432eff03e76d9,254898,sync7272,10,False,False,其實我哋可唔可以集體起訴班狗\n而家一句洗黑錢就收曬我地皮\n又話8000萬捐款得2000萬...,13,1,1576861969,None
1,False,1791346,1026f622b11d1b81b68fe1ce3a64da6d660d3689,254898,sync7272,10,False,False,捐錢為手足提供法律授助係合法架,2,0,1576862115,254898
2,False,1791346,ccdc1cdc9cc1e8a5d1be888e4a2d56daf95926ad,254898,sync7272,10,False,False,沒有黑錢，只有黑警,1,0,1576862209,254898
3,False,1791346,d095a6c5ca7b33ea5015c9a2a8c11623f77c2a2e,254898,sync7272,10,False,False,沒有黑錢，只有黑警,2,0,1576862459,254898
4,False,1791346,2758e161ef0ab437dc49afa8203f29ad61074839,327740,支爆女神,10,True,False,【习近平之春】知唔知 點解連續兩日 有突發事件 令連登洗版？\nhttps://lih.kg...,0,0,1576862502,254898


In [ ]:
import networkx as nx
import json

G = nx.DiGraph()

# with open('../scraper/posts.json') as f:
#   posts = json.load(f)

for index, post in progressbar.progressbar(posts_data.iterrows()):
    # print(post)
    if post['root'] is True:
        G.add_node(post['user_id'])
    else:
        if post['is_not_push_post'] is True:
            G.add_edge(post['user_id'], post['reply_to'], weight=0.3)
        else:
            G.add_edge(post['user_id'], post['reply_to'], weight=1)
            
hub, aut = nx.hits(G)

ac = nx.eigenvector_centrality(G)
cluster = nx.clustering(G)
betweenness = nx.betweenness_centrality(G)

nx.write_gexf(G, "lihkg.gexf")

| |                                          #  | 4003958 Elapsed Time: 0:09:04


In [ ]:
hub = {int(k):v for k,v in hub.items()}
aut = {int(k):v for k,v in aut.items()}

ac = {int(k):v for k,v in ac.items()}
cluster = {int(k):v for k,v in cluster.items()}
betweenness = {int(k):v for k,v in betweenness.items()}

posts_data['hub_score'] = posts_data['user_id'].map(hub)
posts_data['aut_score'] = posts_data['user_id'].map(aut)
posts_data['eigenvector_centrality'] = posts_data['user_id'].map(ac)
posts_data['clustering'] = posts_data['user_id'].map(cluster)
posts_data['betweenness_centrality'] = posts_data['user_id'].map(betweenness)

KNOWN_TROLLS = ['41853','194398','76776','71393','30019','37596','63950','310982','282494','72457','326966','322121','322637','121095','169','213597','226126','245201','159448','149494','181778','324892','273582','118226','240250','316329','77788','219767','71341','12186','228352','75196','28435','25002','247732','322304','258598','133525','14941','27416','222907','48631','158008','289946','96230','40470','186800','30788','288523','254191','84585','85242','123870','89514','170281','103799','64699','49699','299923','71140','122676','61748','202464','288351','234658','277044','149978','6237','98232','69484','165028','328913','132380','146030','331780','266100','52773','275585','249366','41351','273474','39751','51615','326708','213952','1210','270563','241151','70493','335209','276916','232237','94819','78468','203248','100028','291185','95649','332154','83060','264328','244624','105890','5591','171664','23721','129619','326414','14965','51985','191271','253253','25771','153956','32583','72825','336250','58089','70734','124277','173333','155597','137604','276602','41818','162674','151378','295632','147880','314774','57035','88029','97104','216114','64295','28833','203010','264400','273157','238973','57405','28870','132128','199261','126609','65530','172349','173849','192248','261633','127675','34757','166655','228589','343059','144849']
KNOWN_TROLLS = list(map(int, KNOWN_TROLLS))

posts_data['troll'] = posts_data['user_id'].apply(lambda x: 0 if (x not in KNOWN_TROLLS) else 1)

posts_data['root'] = posts_data['root'].apply(lambda x: 1 if x else 0)
posts_data['is_newbie'] = posts_data['is_newbie'].apply(lambda x: 1 if x else 0)
posts_data['is_not_push_post'] = posts_data['is_not_push_post'].apply(lambda x: 1 if x else 0)

posts_data = posts_data.sample(frac=1).reset_index(drop=True)
posts_data = posts_data.drop(['user_id'], axis=1)

#posts_data

In [ ]:
posts_data[posts_data['troll'] == 1]

In [ ]:
train, test = train_test_split(posts_data, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

In [ ]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('troll')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [ ]:
feature_columns = []

# numeric cols
for header in ['user_level', 'like_count', 'dislike_count', 'hub_score', 'aut_score', 'eigenvector_centrality', 'clustering', 'betweenness_centrality']:
  feature_columns.append(feature_column.numeric_column(header))

feature_columns.append(feature_column.indicator_column(feature_column.categorical_column_with_vocabulary_list(
      'is_newbie', [0, 1])))

feature_columns.append(feature_column.indicator_column(feature_column.categorical_column_with_vocabulary_list(
      'is_not_push_post', [0, 1])))

feature_columns.append(feature_column.indicator_column(feature_column.categorical_column_with_vocabulary_list(
      'root', [0, 1])))


In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [ ]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of targets:', label_batch )

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.4),
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.4),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['acc'])

history = model.fit(train_ds,
          validation_data=val_ds,
          epochs=100, callbacks=[early_stop])

In [ ]:
loss, acc = model.evaluate(test_ds)
print("Accuracy", accuracy)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()